# Evaluation with sample users

In the evaluation, we model three types of users:
- Uninformed user (the opposite of nobre)
- 50-50 user (50% of the time aligned with nobre)
- Informed user (fully aligned with nobre scores)

Given the set of system predicates to start off with, we'll see how the system responds to the user feedback. This notebook performs the evaluation

In [1]:
import json
from scipy.stats import spearmanr, kendalltau
import altair as alt
import pandas as pd
from vigor import VIGOR, Visualization, predicates, Predicate
from vigor.visualization_types import VisualizationType

In [2]:
def calculate_correlations(file_path, label):
    vigor = VIGOR()

    visualization_preds = {}
    for vis, score, stat, min_val, max_val in predicates:
        if vis not in visualization_preds:
            visualization_preds[vis] = Visualization(vis)
        visualization_preds[vis].add_predicate(Predicate(stat, min_val, max_val, score))

    for vis in visualization_preds.values():
        vigor.add_visualization(vis)

    with open(file_path, 'r') as file:
        data = json.load(file)

    correlation_data = []

    for interaction_index, interaction in enumerate(data[:5]):
        statistics = interaction['statistics']
        sorted_visualizations = interaction['sorted_visualizations']
        recommendations_with_scores = vigor.recommend(statistics, 8)  # Get recommendations with scores

        print("recommendations_with_scores", recommendations_with_scores)
        for i, (recommendation, score) in enumerate(recommendations_with_scores):
            recommendation_str = recommendation.name.upper()  # Convert to match the format in sorted_visualizations

            if recommendation_str in sorted_visualizations:
                position = sorted_visualizations.index(recommendation_str)
                feedback = 1 / (position + 1)  # Higher feedback for better positions
            else:
                feedback = 0  # If not found in sorted_visualizations

            print("feedback", feedback)
            # Update visualization with feedback based on the recommendation
            for vis in vigor.visualizations:
                if vis.visualization_type == recommendation:
                    print(vis, statistics)
                    print(vis.predicates)
                    vis.update(feedback, statistics)  # Update the visualization with feedback
                    break

        # Calculate Spearman's rank correlation
        min_len = min(len(recommendations_with_scores), len(sorted_visualizations))
        
        # Truncate recommendations and scores
        recommendations_truncated = recommendations_with_scores[:min_len]
        sorted_visualizations_truncated = sorted_visualizations[:min_len]

        recommendations_indices = [list(VisualizationType).index(rec[0]) for rec in recommendations_truncated]  # Get indices of recommended visualizations
        sorted_visualizations_names = [vis.name.upper() for vis in VisualizationType]
        sorted_visualizations_indices = [sorted_visualizations_names.index(visualization) for visualization in sorted_visualizations_truncated]

        # Calculate metrics
        spearman_corr = spearmanr(recommendations_indices, sorted_visualizations_indices).statistic
        kendall_corr = kendalltau(recommendations_indices, sorted_visualizations_indices).statistic

        # Store the results
        correlation_data.append({
            'interaction_index': interaction_index,
            'correlation': spearman_corr,
            'kendall': kendall_corr,
            'label': label
        })

    return correlation_data

In [3]:
# Calculate correlations for three different JSON files
correlation_data_1 = calculate_correlations('../data/evaluation/uninformed_user.json', 'Uninformed User')
correlation_data_2 = calculate_correlations('../data/evaluation/informed_user.json', 'Informed User')
correlation_data_3 = calculate_correlations('../data/evaluation/fifty_fifty_user.json', 'Balanced User')

recommendations_with_scores [(<VisualizationType.BIOFABRIC: 'BioFabric'>, np.float64(0.8341052631578947)), (<VisualizationType.QUILTS: 'Quilts'>, np.float64(0.767438596491228)), (<VisualizationType.ADJACENCY_MATRIX: 'Adjacency Matrix'>, np.float64(0.7507719298245613)), (<VisualizationType.NODE_LINK: 'Node-Link'>, np.float64(0.6660350877192982)), (<VisualizationType.ATTRIBUTE_DRIVEN_NODELINK: 'Attribute Driven Positioning'>, np.float64(0.5993684210526316)), (<VisualizationType.SEMANTIC_SUBSTRATES: 'Semantic Substrates'>, np.float64(0.5993684210526316)), (<VisualizationType.TREEMAP: 'Treemap'>, np.float64(0.2827017543859649)), (<VisualizationType.SUNBURST: 'Sunburst'>, np.float64(0.2827017543859649))]
feedback 0.16666666666666666
<vigor.visualization.Visualization object at 0x3165dcca0> {'size': 41, 'density': 0.05, 'volume': 40, 'node_types': 1, 'node_attributes': 6, 'edge_types': 1, 'edge_attributes': 1}
[<vigor.predicate.Predicate object at 0x3165dcc70>, <vigor.predicate.Predicate obj

KeyError: 'is_bipartite'

In [ ]:
all_correlation_data = correlation_data_1 + correlation_data_2 + correlation_data_3
correlation_df = pd.DataFrame(all_correlation_data)

chart = alt.Chart(correlation_df).mark_line(point=True).encode(
    x='interaction_index:Q',
    y='correlation:Q',
    color=alt.Color('label:N', legend=alt.Legend(title="Dataset", orient='top-left')),  # Legend inside the graph
    tooltip=['interaction_index:Q', 'correlation:Q', 'label:N']
).properties(
    title='Spearman Rank Correlation Over Time',
    width=600,
    height=400
).configure_view(
    strokeOpacity=0
).configure_background(
    color='transparent'
)

chart.display()